In [13]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import folium
from folium import plugins

In [14]:
df = pd.read_csv('populated_bird_data.csv')
target_bird = 'Blackcap'
df = df[df['species'] == target_bird]
species_list = [target_bird]
print(f"Processing single species: {target_bird}")
m = folium.Map(location=[20, 15], zoom_start=3, tiles='CartoDB dark_matter')

Processing single species: Blackcap


In [15]:
def get_smooth_path(df_subset):
    if df_subset.empty:
        return []
    df_subset = df_subset.sort_values('day_of_year')
    df_subset['week'] = df_subset['day_of_year'] // 7
    weekly = df_subset.groupby('week')[['latitude', 'longitude']].mean().reset_index()
    return list(zip(weekly['latitude'], weekly['longitude']))

In [16]:
feature_cols = ['year', 'day_of_year', 'ndvi', 'ndwi', 'ndmi', 'ndbi', 'mndwi']
target_cols = ['latitude', 'longitude']

for bird_name in species_list:
    print(f"  - Training model for: {bird_name}")
    data = df[df['species'] == bird_name].copy()
    data['date'] = pd.to_datetime(data[['year', 'month', 'day']])
    data['day_of_year'] = data['date'].dt.dayofyear
    data_clean = data.dropna(subset=feature_cols + target_cols)

    if len(data_clean) < 10:
        print(f"    * Skipping {bird_name} (Not enough data points)")
        continue

    X = data_clean[feature_cols]
    y = data_clean[target_cols]
    
    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X, y)
    future_days = np.arange(1, 366)
    X_future = pd.DataFrame({'year': [2030]*365, 'day_of_year': future_days})
    avg_indices = data_clean.groupby('day_of_year')[['ndvi', 'ndwi', 'ndmi', 'ndbi', 'mndwi']].mean()
    X_future = X_future.merge(avg_indices, on='day_of_year', how='left')
    X_future = X_future.interpolate(method='linear').bfill().ffill()
    X_future = X_future[feature_cols]
    pred_coords = model.predict(X_future)   
    future_data = X_future.copy()
    future_data['latitude'] = pred_coords[:, 0]
    future_data['longitude'] = pred_coords[:, 1] 
    path_2016 = get_smooth_path(data[data['year'] == 2016].copy())
    path_2024 = get_smooth_path(data[data['year'] == 2024].copy())
    path_2030 = get_smooth_path(future_data.copy())
    fg = folium.FeatureGroup(name=bird_name)
    
    if path_2016:
        folium.PolyLine(
            path_2016, color='#00FFFF', weight=2, opacity=0.6, 
            tooltip=f'{bird_name} 2016 (Actual)'
        ).add_to(fg)
        
    if path_2024:
        folium.PolyLine(
            path_2024, color='#FF00FF', weight=3, opacity=0.8, 
            tooltip=f'{bird_name} 2024 (Actual)'
        ).add_to(fg)
        
    if path_2030:
        folium.PolyLine(
            path_2030, color='#32CD32', weight=4, opacity=0.9, 
            dash_array='5, 10', tooltip=f'{bird_name} 2030 (Predicted)'
        ).add_to(fg)
        folium.CircleMarker(
            path_2030[0], radius=5, color='#32CD32', fill=True, 
            popup=f'{bird_name} 2030 Start'
        ).add_to(fg)

    fg.add_to(m)

  - Training model for: Blackcap


In [17]:
legend_html = f'''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 180px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:rgba(255, 255, 255, 0.8);
     padding: 10px; border-radius: 5px;">
     <b>{target_bird} Migration</b><br>
     <i style="color:#00FFFF">___</i> 2016 (Actual)<br>
     <i style="color:#FF00FF">___</i> 2024 (Actual)<br>
     <i style="color:#32CD32">_ _</i> 2030 (Predicted)<br>
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

In [18]:
m.save("sample.html")
print("Map generated: sample.html")

Map generated: sample.html
